In [1]:
import numpy as np
from scipy.special import erf
from numpy import pi, abs, max, sqrt, cos, arccos, sin, tan, exp

from numpy.linalg import inv, eig
from numpy.fft import fft, ifft

import matplotlib.pyplot as plt

In [2]:
def FourierD(N):
    row = np.arange(N)[:, None]
    col = np.arange(N)[None, :]
    x = 2 * pi * row / N
    D = (row != col) * 0.5 * (-1)**(row + col) / tan(0.5 * (x[row, 0] - x[col, 0]) + (row == col))
    return x, D

def ChebyshevD(N):
    row = np.arange(N + 1)[:, None]
    col = np.arange(N + 1)[None, :]
    x = np.cos(np.pi * row / N)
    c = ((1 + (col % N == 0)) * (-1) ** col).T
    X = np.ones(N + 1) * x
    dX = X - X.T
    D = c @ (1 / c.T) / (dX + np.eye(N + 1))
    D = D - np.diag(np.sum(D, axis=1))
    return x, D

def Laplacian(N, M):
    r, D1r = ChebyshevD(N)
    t, D1t = FourierD(M)
    D2r    = D1r @ D1r
    D2t    = D1t @ D1t
    
    D1rp = D1r[1:N//2 + 1, 1:N//2 + 1]   # First radial derivative, positive r
    D1rn = D1r[1:N//2 + 1, N-1:N//2:-1]  # First radial derivative, negative r
    D2rp = D2r[1:N//2 + 1, 1:N//2 + 1]   # Second radial derivative, positive r
    D2rn = D2r[1:N//2 + 1, N-1:N//2:-1]  # Second radial derivative, negative r
    
    R = np.diag(1/r[1:N//2 + 1, 0])
    IZ = np.concatenate([np.eye(M//2), np.zeros((M//2, M//2))], axis = 1)
    ZI = np.concatenate([np.zeros((M//2, M//2)), np.eye(M//2)], axis = 1)
    T1 = np.concatenate([IZ, ZI], axis = 0)
    T2 = np.concatenate([ZI, IZ], axis = 0)
    
    L = np.kron(D2rp + R @ D1rp, T1) + np.kron(D2rn + R @ D1rn, T2) + np.kron(R @ R, D2t)  # Laplacian operator

    return L

In [4]:
N = 35
M = 20
lam, V = eig(-Laplacian(N, M))
lam = np.sort(sqrt(lam.real))
lam 

array([  2.40482556,   2.40482556,   3.83170597,   3.83170597,
         5.1356223 ,   5.1356223 ,   5.52007811,   5.52007811,
         6.3801619 ,   6.3801619 ,   7.01558667,   7.01558667,
         7.58834243,   7.58834243,   8.41724414,   8.41724414,
         8.65372791,   8.65372791,   8.77148382,   8.77148382,
         9.76102313,   9.76102313,   9.93610952,   9.93610952,
        10.17346814,  10.17346814,  11.06470949,  11.06470949,
        11.08637002,  11.08637002,  11.61984117,  11.61984117,
        11.79153444,  11.79153444,  12.22509226,  12.22509226,
        12.3386042 ,  12.3386042 ,  13.01520072,  13.01520072,
        13.32369194,  13.32369194,  13.35430048,  13.35430048,
        13.58929017,  13.58929017,  14.37253667,  14.37253667,
        14.79595178,  14.79595178,  14.82126873,  14.82126873,
        14.93091771,  14.93091771,  15.70017408,  15.70017408,
        16.03777419,  16.03777419,  16.22346616,  16.22346616,
        16.47063005,  16.47063005,  17.00381967,  17.00